In [ ]:
%run visualisation.ipynb
%run read_graph.ipynb

g = read_nba_full("../Scrapers/BasketballReferenceScraper/data/Players.csv",
                  "../Scrapers/BasketballReferenceScraper/data/Edges.csv",
                  False, True)

all181920 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588]
nba = g.subgraph(all181920)

In [ ]:
result = nba.community_label_propagation()
resultArray = result.membership
i = 0
for v in nba.vs:
    v["communities"] = [resultArray[i]]
    i += 1

plot_full(nba, "lpa_allstars_181920.svg", 1)